## CS431/631 Data Intensive Distributed Analytics
### Winter 2022 - Assignment 2
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Clement Wu
* **ID:** 20775633

---
#### Overview
For this assignment, you will be using Python and Spark to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) and Python tokenizer module, `simple_tokenize.py`, that you used for the first two assignments.    You will also use the same definition of PMI that was used for [Assignment 1](https://www.student.cs.uwaterloo.ca/~cs451/assignment1-431.html).

We first need to install Apache Spark. Run the next block to download and install Spark. It will take about a minute to finish the instalation.

In [9]:
!apt-get update -qq > /dev/null
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz
!tar xf spark-2.4.8-bin-hadoop2.7.tgz
!pip install -q findspark

To use Spark from within a Python program, it is first necessary to tell the Python interpreter where the Spark installation is located. The code in the following cell tells Python how to find this Spark installation. This code creates SparkContext (sc) for you. Do not change this block.

In [10]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

Once Python knows where Spark is located, you can create a `SparkContext`.   All Spark commands must run within an active `SparkContext`.   The code below will create a `SparkContext`, and store a reference to the context in the variable `sc`. 
The `appName` parameter assigns a name of your choosing to the Spark jobs that are created in this context - this is useful mostly for debugging.   The `master` parameter indicates that Spark jobs will run in local mode, using two threads.   This means that your Spark jobs are not really running on a cluster, and are instead running in a single process on the local machine.   You program Spark jobs the same way whether they run in local mode or on a cluster - the main difference between local and cluster modes is, of course, performance.

Next, let's test that your `SparkContext` has been set up properly by running some simple test code.   This code uses a single Spark job to estimate the value of Euler's number $e$. One way to calculate $e$ is to use the following serries by Jacob Bernoulli:

$p_n = 1 - \frac{1}{1!} + \frac{1}{2!} - \frac{1}{3!} + \cdots + \frac{(-1)^n}{n!} = \sum_{k = 0}^n \frac{(-1)^k}{k!}$

As n tends to infinity, $p_n$ approaches $1/e$.

In the following code,  `parallelize()` and `map()` are Spark *transformations*, and `reduce()` is a Spark *action*.   Study the code in the cell below, then go ahead and run it.   It should take several seconds, since a Spark job is being created and executed, and should print an estimate of $e$ when it finishes.   


In [12]:
import math

n= 10000
inverse_e = sc.parallelize(range(0, n)).map(lambda x: ((-1)**x) * (1 / math.factorial(x))).reduce(lambda x,y:x+y)
e = 1 / inverse_e
print("e = ", e)

e =  2.718281828459044


---
#### Question 1  (4/30 marks):

In the following cell, briefly explain how the example works.   What is the Spark job doing, and how is it used to estimate the value of $e$? How accurate is our estimate?

#### Your answer to Question 1:

First we create a python collection (or a list of 10000 elements) and form an RDD. Then using map fucntion with spark, we calculate each term in Pn equation and transform the content in rdd into elements of terms. By using reduce funtion with spark, we apply specific operation on all elements in rdd, in this case, is addition. Therefore, reduce function adds all terms together, which exactly matches the equation of Pn. Then, we take inverse, we get an estimate of e. In this program, spark helps to calculate all terms in data-parallel fashion. Since we take a large n, the estimate is pretty good. 

---
### Important

###### The questions that follow ask you to implement functions whose prototypes are given to you. Do **NOT** change the prototypes of the functions. Do **NOT** write code outside of the functions. All necessary code should be included in the function body (except for import statements). You may declare functions inside of the function body. When marking, we will execute your code by calling the functions from an external program, which is why your code cannot rely on statements running outside functions. Please remove any call to the functions that you may have introduced for test purposes before submitting your notebook.

---
#### Question 2  (4/30 marks):

Now it is your turn to write some Spark programs. Start with the simple task of counting the number of *distinct* tokens which appear in `Shakespeare.txt`.   You have already written Python code to do this in Assignment 1, but for this assignment we want you to use Spark to solve the same problem.   You should compare the answer you get using Spark with the answer you got from your pure-Python solution from Assignment 1.   Both answers should, of course, be the same.

Run the next block to download the text file (Shakespeare.txt) and the Python tokenizer module (simple_tokenize.py).

In [1]:
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/Shakespeare.txt
!wget -q https://student.cs.uwaterloo.ca/~cs451/content/cs431/simple_tokenize.py

Your code should use Spark, not the Python driver code, to read `Shakespeare.txt` and do the counting.   The idea is to use Spark to give you a data-parallel alternative to the sequential Python solution you wrote for Assignment 1.

Write your solution for question 2 in the code cell below, by implementing the `count_distinct_tokens` function. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_tokens` must return the number of distinct tokens.

In [13]:
from simple_tokenize import simple_tokenize

# Returns the count of distinct tokens in the `Shakespeare.txt` dataset
def count_distinct_tokens():
  # read a text file from local system
  textFile = sc.textFile('Shakespeare.txt')

  # apply tokenization to every line and then count distinct elements
  count = textFile.flatMap(simple_tokenize).distinct().count()
  return count

count_distinct_tokens()
    

25975

---
#### Question 3  (4/30 marks):

Next, write a Spark program that will count the number of distinct token pairs in `Shakespeare.txt`, as you did in Assignment 1. Again, ensure that the answer that you get using Spark matches the answer you got in the first assignment.

Write your solution for question 3 by implementing the `count_distinct_pairs` function in the code cell below.   It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `count_distinct_pairs` must return the number of distinct token pairs.

In [14]:
from simple_tokenize import simple_tokenize

# this program returns all pairs of elements in the list
def pairs(alist):
  return [(i, j) for i in alist for j in alist if i!=j]

# Returns the count of distinct pairs in the `Shakespeare.txt` dataset
def count_distinct_pairs():
  # read a text file from a local system
  textFile = sc.textFile('Shakespeare.txt').cache()
  
  # apply tokenization first, then transform each line into lists of pairs, 
  # count all distinct pairs after that
  count = textFile.flatMap(lambda l : pairs(simple_tokenize(l))).distinct().count()
  
  return count

count_distinct_pairs()
    

1969760

---
#### Question 4  (6/30 marks):

Next, write Spark code that will calculate the probability $p(x)$ (as defined in Assignment 1) for every distinct token $x$ in `Shakespeare.txt`.   Your function should return the 50 highest-probability tokens, and their counts and probabilities.

Make sure that your solution calculates probabilities and identifies the 50 highest-probability tokens in a data-parallel fashion, using Spark transformations and actions.   Only the 50 highest-probability tokens (and their counts and probabilities) should be returned by Spark to your driver code.

Write your solution for question 4 by implementing the `top_50_tokens_probabilities` function in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `top_50_tokens_probabilities` must return a list of 50 (probability, count, token) tuples, ordered by probability, that is, the list returned by the function should be of the form: `[(proba1, count1, token1), (proba2, count2, token2), ..., (proba50, count50, token50)]`

In [15]:
from simple_tokenize import simple_tokenize

# Returns a list of the top 50 (probability, count, token) tuples, ordered by probability
def top_50_tokens_probabilities():
  # read a text file from local system
  textFile = sc.textFile('Shakespeare.txt')

  # count total lines in the file  
  total = textFile.cache().count()

  # take all distinct element in every line, then map it to tuple of token, count and probability
  # after that apply reduce operation to sum up all count and probibility by keys
  summary = textFile.flatMap(lambda l : list(set(simple_tokenize(l)))).map(lambda word : (word, (1, 1/total))).reduceByKey(lambda x, y : (x[0]+y[0], x[1]+y[1]))

  # flat map all tuples and rearrange elements, sort by probability in descending order, take first 50 tuples
  return summary.map(lambda t : (t[1][1], t[1][0], t[0])).sortBy(lambda x: x[0], ascending=False).cache().take(50)

top_50_tokens_probabilities()




[(0.20091786571727055, 24604, 'and'),
 (0.1984353819269089, 24300, 'the'),
 (0.15235427656832087, 18657, 'i'),
 (0.14892452922637386, 18237, 'to'),
 (0.13575266622027737, 16624, 'of'),
 (0.10844534452629928, 13280, 'a'),
 (0.09959332995803605, 12196, 'you'),
 (0.09430988583841768, 11549, 'my'),
 (0.08667461497003565, 10614, 'in'),
 (0.08630714204054132, 10569, 'that'),
 (0.07150206601447007, 8756, 'is'),
 (0.06720671577193643, 8230, 'not'),
 (0.06167012363422232, 7552, 'with'),
 (0.06039621747864281, 7396, 'me'),
 (0.05982459292165162, 7326, 'for'),
 (0.05836286726877482, 7147, 'it'),
 (0.0544023256953377, 6662, 'be'),
 (0.052466968266668426, 6425, 'this'),
 (0.05228731483447127, 6403, 'his'),
 (0.05089908376749332, 6233, 'your'),
 (0.050670433944696947, 6205, 'but'),
 (0.04749383462084737, 5816, 'he'),
 (0.046889545803456965, 5742, 'have'),
 (0.04132028940534538, 5060, 'thou'),
 (0.04015254209606392, 4917, 'as'),
 (0.0395237550833739, 4840, 'him'),
 (0.03949109082297442, 4836, 'so'),


---
#### Question 5  (6/30 marks):

Next, write the code for the function `PMI` that will take a positive integer threshold $T$ as an argument, and then return all token pairs that co-occur at least $T$ times in `Shakespeare.txt`.   For each such pair $(x,y)$, the function should also return PMI$(x,y)$, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears. You can compare the results produced by this code with the results of Two-Token queries (from Assignment 1) for consistency.

Write your solution for question 5 by implementing the function `PMI` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI` should return a list of ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count) tuples, that is, the list returned by the function should be of the form: `[((token1, token2), pmi, cooc_count, token1_count, token2_count), (...), ((other_token1, other_token2), other_pmi, other_cooc_count, other_token1_count, other_token2_count)]`.


As always, calculations should be done in data-parallel fashion by using Spark.

In [11]:
from simple_tokenize import simple_tokenize
from math import log

# This program returns all pairs of elements in a list
def pairs(alist):
  return [(i, j) for i in alist for j in alist if i!=j]

# This program returns pmi values based on joint probability, probability of x,
# probability of y
def calculation(pxy, px, py):
  return log(pxy/(px*py), 10)

# Returns a list of tuples with the following format:
# ((token1, token2), pmi, co-occurrence_count, token1_count, token2_count)
def PMI(threshold):
  # read text file from local system
  textFile = sc.textFile('Shakespeare.txt')

  # get line counts of this text file
  total = textFile.cache().count()

  # generate pairs of distinct elements in every line
  all_pair = textFile.flatMap(lambda l : list(set(pairs(simple_tokenize(l)))))

  # summarize single token freqency and tranform them into map
  single = textFile.flatMap(lambda l : list(set(simple_tokenize(l)))).map(lambda w : (w, 1)).reduceByKey(lambda x, y : x + y).collectAsMap()

  # summarize pairs of elements with probability and count
  pair_summary = all_pair.map(lambda pair : (pair, (1/total, 1))).reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))

  # filter rdd based on threshold, then add PMI value, count of x, count of y into current rdd
  summary = pair_summary.filter(lambda x : x[1][1] >= threshold).map(lambda x : (x[0], calculation(x[1][0], single[x[0][0]]/total, single[x[0][1]]/total), x[1][1], single[x[0][0]], single[x[0][1]])).cache()
  return summary.collect()

PMI(318)


[(('but', 'as'), 0.10597228806313513, 318, 6205, 4917),
 (('as', 'but'), 0.10597228806313513, 318, 4917, 6205),
 (('time', 'the'), 0.26046525340439214, 381, 1054, 24300),
 (('the', 'time'), 0.26046525340439214, 381, 24300, 1054),
 (('by', 'the'), 0.11866504544635256, 949, 3639, 24300),
 (('the', 'by'), 0.11866504544635256, 949, 24300, 3639),
 (('the', 'that'), 0.0694586044071933, 2461, 24300, 10569),
 (('that', 'the'), 0.0694586044071933, 2461, 10569, 24300),
 (('the', 'and'), 0.045934991833045884, 5427, 24300, 24604),
 (('to', 'the'), 0.051235259829883105, 4072, 18237, 24300),
 (('the', 'to'), 0.051235259829883105, 4072, 24300, 18237),
 (('and', 'to'), 0.017522588038683166, 3815, 24604, 18237),
 (('to', 'and'), 0.017522588038683166, 3815, 18237, 24604),
 (('and', 'the'), 0.045934991833045884, 5427, 24604, 24300),
 (('the', 'or'), -0.041798121420827734, 412, 24300, 2286),
 (('the', 'be'), -0.09797128011614956, 1055, 24300, 6662),
 (('be', 'the'), -0.09797128011614956, 1055, 6662, 24300

---
#### Question 6  (6/30 marks):

Finally, write Spark code for the function `PMI_one_token`, that will take a positive integer threshold $T$ and a sample size $N$ as arguments. For every token $x$ in `Shakespeare.txt`, your code should find all tokens $y$ that co-occur with $x$ at least $T$ times, as well as PMI$(x,y)$ for each such pair, the co-occurrence count of the pair, the number of times $x$ appears, and the number of times $y$ appears.

For each $x$, the output of your program should be equivalent to the output that would be produced by a One-Token query on $x$ (see Assignment 1), with threshold $T$. Rather than producing output for all possible tokens $x$, the function should produce output only for $N$ different $x$'s, chosen at random from among all distinct tokens in the input file.

Write your solution for question 6 by implementing the function `PMI_one_token` in the code cell below. It should use the `SparkContext` which was created previously (referenced by the variable `sc`). The function `PMI_one_token` should return a list of $N$ tuples of the form `(token, [ list_of_cooccurring_tokens ])`, where each entry in `list_of_cooccurring_tokens` is of the form `((token1, token2), pmi, cooc_count, token1_count, token2_count)`.

For instance, if $N$ = 2 and the randomly selected tokens are "if" and "you", a valid format for the value returned by `PMI_one_token` would be:
```
[('if', [(('if', 'you'), -0.09813466615111513, 1975, 16624, 18237), (('if', 'a'), 0.03856379243802408, 1568, 16624, 10569)]), ('you', [(('you', 'if'), -0.09813466615111513, 1975, 18237, 16624)])]
```

Hint: Sampling must be done at the very last step. Hint: there is an action that returns a sample subset from an RDD.


In [ ]:
from simple_tokenize import simple_tokenize
from math import log

# This program returns all pairs of elements in a list
def pairs(alist):
  return [(i, j) for i in alist for j in alist if i!=j]

# This program calculates pmi value based on joint probability, probability of x
# and probability of y
def calculation(pxy, px, py):
  return log(pxy/(px*py), 10)

# Returns a list of samp_size tuples with the following format:
# (token, [ list_of_cooccurring_tokens ])
# where list_of_cooccurring_tokens is of the form
# [((token1, token2), pmi, cooc_count, token1_count, token2_count), ...]
def PMI_one_token(threshold, samp_size):
  # read text file from local system
  textFile = sc.textFile('Shakespeare.txt')

  # get count of lines of a file
  total = textFile.cache().count()

  # get all pairs of tokens in each line
  all_pair = textFile.flatMap(lambda l : list(set(pairs(simple_tokenize(l)))))

  # get summary of tokens and counts
  words = textFile.flatMap(lambda l : list(set(simple_tokenize(l)))).map(lambda w : (w, 1)).reduceByKey(lambda x, y : x + y)

  # convert summary into dictionary
  single = words.collectAsMap()

  # get summary of pairs of tokens, probability and count
  pair_summary = all_pair.map(lambda pair : (pair, (1/total, 1))).reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))

  # filter summary of pairs based on threshold
  summary = pair_summary.filter(lambda x : x[1][1] >= threshold)

  # choose n words based on sample size
  choosed = list(map(lambda x : x[0], words.takeSample(False, samp_size)))

  result = []

  # iterate through sample words
  for i in range(len(choosed)):
    # get summary of specific sample word
    subset = summary.filter(lambda x : x[0][0] == choosed[i]).map(lambda x : (x[0], calculation(x[1][0], single[x[0][0]]/total, single[x[0][1]]/total), x[1][1], single[x[0][0]], single[x[0][1]])).collect()
    
    # append the word and all its summary into the list
    result.append((choosed[i],subset))
  return result

PMI_one_token(1, 4)


[('radish',
  [(('radish', 'not'), 0.8715573313273605, 1, 2, 8230),
   (('radish', 'fifty'), 3.281807188219724, 1, 2, 32),
   (('radish', 'if'), 1.2492639721722394, 1, 2, 3449),
   (('radish', 'or'), 1.4278809404803672, 1, 2, 2286),
   (('radish', 'a'), 0.9647890871716127, 2, 2, 13280),
   (('radish', 'were'), 1.5974668528402627, 1, 2, 1547),
   (('radish', 'the'), 0.40135089294131815, 1, 2, 24300),
   (('radish', 'for'), 0.9220902522111042, 1, 2, 7326),
   (('radish', 'all'), 1.204439282935568, 1, 2, 3824),
   (('radish', 'two'), 1.9431017439164693, 1, 2, 698),
   (('radish', 'there'), 1.5382484309387126, 1, 2, 1773),
   (('radish', 'of'), 0.5662216363265281, 1, 2, 16624),
   (('radish', 'them'), 1.5056964794846175, 1, 2, 1911),
   (('radish', 'am'), 1.460621305610879, 1, 2, 2120),
   (('radish', 'bunch'), 4.184897175211667, 1, 2, 4),
   (('radish', 'i'), 0.5161153550064446, 1, 2, 18657),
   (('radish', 'world'), 1.9931667818488117, 1, 2, 622),
   (('radish', 'like'), 1.54119781057235

---
Don't forget to save your workbook!   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.